In [205]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import gc
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore',category = DeprecationWarning)
warnings.simplefilter(action='ignore',category = DeprecationWarning)


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, RepeatedKFold, GroupKFold
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import ClusterCentroids,RandomUnderSampler,NearMiss,EditedNearestNeighbours,RepeatedEditedNearestNeighbours,AllKNN
from imblearn.over_sampling import RandomOverSampler,SMOTE,ADASYN
from imblearn.ensemble import EasyEnsemble,BalanceCascade,BalancedBaggingClassifier
from imblearn.combine import SMOTEENN,SMOTETomek

import category_encoders as ce
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from time import time

import scipy.stats as st
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from shutil import rmtree

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

from sklearn.metrics import make_scorer, confusion_matrix, accuracy_score, f1_score, recall_score, roc_auc_score, cohen_kappa_score

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
import lightgbm as lgb

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate

from skopt import BayesSearchCV
class BayesSearchCV(BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

kappa_scorer = make_scorer(cohen_kappa_score, greater_is_better=True)


In [125]:
df_train = pd.read_hdf('Elo_fe.h5', 'train_fe_3691')
df_test = pd.read_hdf('Elo_fe.h5', 'test_fe_3691')

outliers = df_train['outliers']

df_train = df_train.drop(['first_active_month', 'outliers', 'target'], axis=1)
df_test = df_test.drop(['first_active_month', 'outliers', 'target'], axis=1)

In [126]:
#df_train = pd.read_hdf('Elo_fe.h5', 'df_train')
#df_test = pd.read_hdf('Elo_fe.h5', 'df_test')

#df_train = pd.read_hdf('Elo_fe.h5', 'train_fe_3691')
#df_test = pd.read_hdf('Elo_fe.h5', 'test_fe_3691')

#id_test = pd.read_hdf('Elo_fe.h5', 'card_id')

for col in df_train.columns:
    df_train[col].fillna(0, inplace=True)
    df_test[col].fillna(0, inplace=True)
    
    df_train[col].replace([np.inf,-np.inf], 0, inplace=True)
    df_test[col].replace([np.inf,-np.inf], 0, inplace=True)
    
    if df_train[col].dtypes == 'float16':
        df_train[col] = df_train[col].astype('float32')
        df_test[col] = df_test[col].astype('float32')

X_train, X_val, y_train, y_val = train_test_split(df_train, outliers, test_size=0.3, random_state=1)


In [127]:
df_train.dtypes.value_counts()

float32    115
float64     61
int64       23
dtype: int64

In [214]:
## takes too long

param_search = {
 'base_estimator__boosting_type': ['goss'],
 'base_estimator__colsample_bytree': [0.8],
 'base_estimator__learning_rate': [0.1, 0.15, 0.2],
 'base_estimator__n_estimators': [2000, 6000],
 'base_estimator__num_leaves': [63, 127],
 'base_estimator__objective': ['binary'],
 'base_estimator__random_state': [501],
 'base_estimator__reg_alpha': [1.6, 2],
 'base_estimator__reg_lambda': [1.6, 2],
 'base_estimator__subsample': [0.7],
 'base_estimator__is_unbalance': [True],
 'max_samples': [0.1, 0.5, 1.0],
 'n_estimators': [2000, 6000]
}

grid_clf = GridSearchCV(
        BalancedBaggingClassifier(base_estimator=lgb.LGBMClassifier(n_thread=-1),
                                  ratio='auto',
                                  replacement=False,
                                  random_state=0),
        param_search, scoring = kappa_scorer,
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42),   
)

In [215]:
grid_clf.fit(df_train, outliers)


KeyboardInterrupt: 

In [129]:
thres = 676/(59900 + 676)

y_pred_proba = bbc.predict_proba(X_val)
y_pred_proba = pd.DataFrame(y_pred_proba)

In [69]:
# set probability threshold = 0.01159

In [130]:
y_pred_proba['prediction'] = 0
y_pred_proba['prediction'][y_pred_proba[0]<thres] = 1
y_pred_proba['actual'] = y_val

/Users/jincui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [131]:
confusion_matrix(y_val, y_pred_proba['prediction'])

array([[59500,   400],
       [  460,   216]])

In [132]:
cohen_kappa_score(y_val, y_pred_proba['prediction'])

0.3272059279341084

In [133]:
confusion_matrix(y_val, y_pred1)

array([[51818,  8082],
       [   17,   659]])

In [139]:
rus = RandomUnderSampler(random_state=0)


In [ ]:
rus.fit_sample

In [146]:
rus = RandomUnderSampler(random_state=0)
ros = RandomOverSampler(random_state=0)

X_rus, y_rus = rus.fit_sample(df_train, outliers)
X_ros, y_ros = ros.fit_sample(df_train, outliers)

X_rus_train, X_rus_val, y_rus_train, y_rus_val = train_test_split(X_rus, y_rus, test_size=0.3, random_state=1)
X_ros_train, X_ros_val, y_ros_train, y_ros_val = train_test_split(X_ros, y_ros, test_size=0.3, random_state=1)



In [169]:
X_rus_val.shape

(1325, 199)

In [170]:
df_test.shape

(123623, 199)

In [147]:
clf_pre = LinearSVC().fit(X_rus_train, y_rus_train)

/Users/jincui/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [203]:
ITERATIONS = 10 # 1000
TRAINING_SIZE = 100000 # 20000000
TEST_SIZE = 25000

bayes_cv_tuner = BayesSearchCV(
    
    estimator = 
        xgb.XGBClassifier(
            n_jobs = 1,
            objective = 'binary:logistic',
            eval_metric = 'roc',
            silent=1,
            tree_method='approx',
            
        ), 

    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = kappa_scorer,
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):    
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    

In [202]:
result = bayes_cv_tuner.fit(df_train, outliers, callback=status_print)

XGBoostError: b'[06:32:48] src/metric/metric.cc:21: Unknown metric function make_scorer(cohen_kappa_score)\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.dylib                    0x000000011fc7a120 dmlc::StackTrace() + 288\n[bt] (1) 1   libxgboost.dylib                    0x000000011fc79ebf dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x000000011fcee826 xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) + 886\n[bt] (3) 3   libxgboost.dylib                    0x000000011fc749be xgboost::LearnerImpl::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<c

In [163]:
result.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree',
       colsample_bylevel=0.4503841871781403,
       colsample_bytree=0.9195352964526833, eval_metric='auc',
       gamma=8.168958221061441e-09, learning_rate=0.07356404539935663,
       max_delta_step=4, max_depth=23, min_child_weight=1, missing=None,
       n_estimators=88, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0,
       reg_alpha=0.00010376808625045426, reg_lambda=476.96194787286544,
       scale_pos_weight=1.3165669602830552, seed=None, silent=1,
       subsample=0.387658500562527, tree_method='approx')

In [164]:
y_rus_pred = result.predict(X_rus_val)
confusion_matrix(y_rus_pred, y_rus_val)

array([[529, 101],
       [139, 556]])

In [179]:
y_rus_pred_proba = result.predict_proba(X_rus_val)
y_rus_pred_proba = pd.DataFrame(y_rus_pred_proba)

In [183]:
cohen_kappa_score(y_rus_pred, y_rus_val)

0.6378833255520002